# Autenticacion

## Flujo de Autenticacion

Ahora empezaremos con el módulo de autenticaciones pero antes quiero explicarte un poco acerca de lo que estaremos realizando en nuestra aplicación y cómo será el proceso de autenticación y autorización.

### Ruta para iniciar sesión

Lo que obtendremos como resultado al final de este módulo es la protección de determinadas rutas de nuestra aplicación para las cuales solo se podrá acceder mediante el inicio de sesión del usuario. Para esto crearemos una ruta que utilice el método POST donde se solicitarán los datos como email y contraseña.

### Creación y envío de token

Luego de que el usuario ingrese sus datos de sesión correctos este obtendrá un token que le servirá para enviarlo al momento de hacer una petición a una ruta protegida.

### Validación de token

Al momento de que nuestra API reciba la petición del usuario, comprobará que este le haya enviado el token y validará si es correcto y le pertenece. Finalmente se le dará acceso a la ruta que está solicitando.

En la siguiente clase empezaremos con la creación de una función que nos va a permitir generar tokens usando la librería **pyjwt**.

pyJWT (Python JSON Web Token) es una biblioteca de Python que se utiliza para codificar y decodificar tokens JWT (JSON Web Token). Un token JWT es un objeto de seguridad que se utiliza para autenticar a los usuarios en aplicaciones web y móviles. Los tokens JWT se emiten por un servidor de autenticación y luego se envían al cliente, que los utiliza para demostrar su identidad al acceder a recursos protegidos en el servidor

## Generando tokens con pyjwt

Se generara un token luego que el usuario inicie seccion en la aplicacion. Dicho token servira para enviarlo a determinadas rutas que esten protegidas en la aplicacion.

Se añade el modulo *pyjwt* a requirementes. Se crea un nuevo archivo: **jwt_manager.py**, donde se implementa la funcion *create_token*

```py
from jwt import encode

def create_token(data:dict):
    token: str = encode(payload=data, key="my_secret_key", algorithm="HS256")
    return token
```
encode tiene varios parametros

- payload: el contenido que se va convertir

### dentro de main.py

Se importa el modulo. Se crea un nuevo modelo que permita añadir informacion del usuario:

```py
class User(BaseModel):
    username: str
    email: str
    password: str
```

Y una nueva ruta que permita el login del usuario:

```py
@app.post('/login', tags=['auth'], status_code=201)
def login(user: User):
    user = dict(user)
    return JSONResponse(content={"message": "welcome user",
                                 "new_user": user["username"]}, status_code=201)
```

Prueba dentro de la documentacion, que la ruta este bien, y no este generando ningun error.

## Validando Tokens

Modificaremos la funcion anterior para si el usuario es *platzi* y su contraseña es *root*. En caso de no estar autenticado devolveremos un 401.

```py
@app.post('/login', tags=['auth'], status_code=201)
def login(user: User):
    if user.username == "platzi" and user.password == "root":
        token: str = create_token(user.dict())
        return JSONResponse(content={"message": "user authorized",
                                 "username": user.username,
                                 "token": token}, status_code=201)

    return JSONResponse(content={"message": "Not Authorized"}, status_code=401) 👈
```
Aqui muestro los dos casos como se ven en la documentacion:

![](https://i.imgur.com/SHlrQLj.png)

![](https://i.imgur.com/zYa5Lk8.png)

### decodificando el token

Vamos a *jwt_manager* e importamos *decode*:

    from jwt import encode, decode

```py
def validate_token(token: str) -> dict:
    data: dict = decode(token, key="my_secret_key", algorithms=['HS256']) 👈
    return data
```

para decodificar el token se necesita la *clave secreta*.